In [19]:
""" 
Imports
"""

import pandas as pd

In [ ]:
import pandas as pd

# Arquivos a serem carregados
path = '../archive/'
file_names = [f"BO_{year}_{part}.csv" for year in range(2007, 2014) for part in range(1, 3)]
file_names += ["BO_2015.csv",'BO_2016.csv']

# Carregar dados do CSV
# pd.read_csv: Lê os arquivos CSV.
# Amostragem (sample): Reduz a dimensão dos dados para facilitar a análise e economizar memória/processamento.
# Concatenação (pd.concat): Junta os DataFrames amostrados em um único conjunto.

def load_and_sample_files(file_names,path, sample_fraction=0.05):
    sampled_dfs = []
    for file in file_names:
        print(file)
        try:
            df = pd.read_csv(path+file,low_memory=False)
            
            sampled_df = df.sample(frac=sample_fraction, random_state=1)
            sampled_dfs.append(sampled_df)
        except FileNotFoundError:
            print(f"File {file} not found.")
    return pd.concat(sampled_dfs, ignore_index=True)

# Arquivos no diretório
sampled_data = load_and_sample_files(file_names,path)

sampled_data.count()


BO_2007_1.csv
BO_2007_2.csv
BO_2008_1.csv
BO_2008_2.csv
BO_2009_1.csv
BO_2009_2.csv
BO_2010_1.csv
BO_2010_2.csv
BO_2011_1.csv
BO_2011_2.csv
BO_2012_1.csv
BO_2012_2.csv
BO_2013_1.csv
BO_2013_2.csv


In [ ]:
sampled_data = sampled_data.loc[:, ~sampled_data.columns.str.contains('^Unnamed')]
sampled_data.dropna()
sampled_data['IDADE_PESSOA'] = pd.to_numeric(sampled_data['IDADE_PESSOA'].str.extract('(\d+)')[0], errors='coerce')
# Save the sampled data into a new CSV file
output_file = "sampled_crime_reports.csv"
sampled_data['DATA_OCORRENCIA_BO'] = pd.to_datetime(sampled_data['DATA_OCORRENCIA_BO'], errors='coerce').dropna()
sampled_data.to_csv(output_file, index=False)
output_file

In [ ]:
sampled_data['CIDADE'].value_counts().head(20)

In [ ]:
sampled_data.columns
[['DESCR_TIPO_PESSOA',
 'IDADE_PESSOA',
 'DESCR_GRAU_INSTRUCAO', 
 'RUBRICA', 
 'CIDADE',
 'SEXO_PESSOA',
 'COR',
 'DESCR_PROFISSAO',
'CONDUTA']]

In [ ]:
import matplotlib.pyplot as plt

def plot_crime_incidents_by_year(data):
    data['ANO_BO'] = data['ANO_BO'].astype(int)  # Ensure the year column is of type int
    incidents_by_year = data['ANO_BO'].value_counts().sort_index()

    plt.figure(figsize=(10, 6))
    incidents_by_year.plot(kind='bar', color='skyblue')
    plt.title('Crime Incidents by Year')
    plt.xlabel('Year')
    plt.ylabel('Number of Incidents')
    plt.xticks(rotation=45)
    plt.show()

plot_crime_incidents_by_year(sampled_data)


In [ ]:
def plot_crime_distribution_by_month(data):
    data['MES'] = data['MES'].astype(int)  # Ensure the month column is of type int
    incidents_by_month = data['MES'].value_counts().sort_index()

    plt.figure(figsize=(10, 6))
    incidents_by_month.plot(kind='bar', color='skyblue')
    plt.title('Crime Distribution by Month')
    plt.xlabel('Month')
    plt.ylabel('Number of Incidents')
    plt.xticks(rotation=45)
    plt.show()

plot_crime_distribution_by_month(sampled_data)


In [ ]:
def plot_crime_categories(data, top_n=10):
    crime_categories = data['RUBRICA'].value_counts().head(top_n)

    plt.figure(figsize=(12, 6))
    crime_categories.plot(kind='bar', color='skyblue')
    plt.title('Top Crime Categories')
    plt.xlabel('Crime Category')
    plt.ylabel('Number of Incidents')
    plt.xticks(rotation=45)
    plt.show()

plot_crime_categories(sampled_data)


In [ ]:
sampled_data[['LATITUDE', 'LONGITUDE', 'ANO_BO', 'MES']].dropna()

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

# Load the dataset
df = pd.read_csv('sampled_crime_reports.csv').dropna()

# Define the columns for clustering
columns_for_clustering = ['DESCR_TIPO_PESSOA', 'IDADE_PESSOA', 'DESCR_GRAU_INSTRUCAO', 'RUBRICA', 
                          'CIDADE', 'SEXO_PESSOA', 'COR', 'DESCR_PROFISSAO', 'CONDUTA']

# Clean and preprocess the data
def clean_and_preprocess_data(data):
    # Select relevant columns and drop rows with NaN values
    data = data[['DESCR_TIPO_PESSOA',
        'IDADE_PESSOA',
        'DESCR_GRAU_INSTRUCAO', 
        'RUBRICA', 
        'CIDADE',
        'SEXO_PESSOA',
        'COR',
        'DESCR_PROFISSAO',
        'CONDUTA']]

    # Clean the 'IDADE_PESSOA' column
    data['IDADE_PESSOA'] = pd.to_numeric(data['IDADE_PESSOA'].str.extract('(\d+)')[0], errors='coerce')

    # Handle missing values by imputing
    imputer = SimpleImputer(strategy='most_frequent')
    data = pd.DataFrame(imputer.fit_transform(data), columns=columns_for_clustering)

    # Convert categorical columns to numeric using LabelEncoder
    label_encoders = {}
    for column in columns_for_clustering:
        if data[column].dtype == 'object':
            label_encoders[column] = LabelEncoder()
            data[column] = label_encoders[column].fit_transform(data[column])
    
    return data, label_encoders

# Apply the cleaning and preprocessing
cleaned_data, label_encoders = clean_and_preprocess_data(df)

# Display the first few rows of the cleaned data
print(cleaned_data.head())


In [ ]:
from sklearn.cluster import KMeans

# Scale the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(cleaned_data)

# Perform KMeans clustering
num_clusters = 3
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
cleaned_data['Cluster'] = kmeans.fit_predict(scaled_data)

# Display the first few rows with the cluster labels
print(cleaned_data.head())


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# Pair plot of selected features colored by cluster
sns.pairplot(cleaned_data, vars=[
        'DESCR_TIPO_PESSOA',
        'IDADE_PESSOA',
        'DESCR_GRAU_INSTRUCAO', 
        'RUBRICA', 
        'CIDADE',
        'SEXO_PESSOA',
        'COR',
        'DESCR_PROFISSAO',
        'CONDUTA'
        ], hue='Cluster', palette='viridis', plot_kws={'alpha':0.6, 's':80})
plt.suptitle('Pair Plot of Clusters', y=1.02)
plt.show()


In [ ]:
# Add the cluster labels to the original dataset
df['Cluster'] = cleaned_data['Cluster']

# Analyze the characteristics of each cluster
def analyze_clusters(data, label_encoders):
    cluster_analysis = data.groupby('Cluster').mean()
    for column in label_encoders:
        cluster_analysis[column] = cluster_analysis[column].map(lambda x: label_encoders[column].inverse_transform([int(x)])[0])
    return cluster_analysis

# Perform the cluster analysis
cluster_analysis = analyze_clusters(cleaned_data, label_encoders)

# Display the cluster analysis
print(cluster_analysis)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Convert the Cluster column to string type for plotting
df['Cluster'] = df['Cluster'].astype(str)

# Plot the distribution of clusters for each RUBRICA
def plot_clusters_for_rubrica(data):
    plt.figure(figsize=(14, 7))
    sns.countplot(data=data, x='RUBRICA', hue='Cluster', palette='viridis')
    plt.title('Distribution of Clusters for Each RUBRICA')
    plt.xlabel('RUBRICA')
    plt.ylabel('Count')
    plt.xticks(rotation=90)
    plt.legend(title='Cluster')
    plt.show()

plot_clusters_for_rubrica(df)


In [ ]:
identity_columns = [
    'TransactionID'
    ,'id_15'
    ,'id_16'
    ,'id_35'
    ,'id_36'
    ,'id_37'
    ,'id_38'
    ,'DeviceType'
    ,'DeviceInfo'
    ,'id_31'
    ,'id_30'
    ,'id_28'
    ,'id_29'
]
df_identity = identity_dataset[identity_columns]
df_identity

In [ ]:
transaction_dataset = pd.read_csv(r'../../train_transaction.csv')
print(transaction_dataset.columns)
transaction_dataset.info()

In [ ]:
"""
- P_ and (R__) emaildomain: purchaser and recipient email domain
- certain transactions don't need recipient, so R_emaildomain is null.

The logic of our labeling is define reported chargeback on the card as fraud transaction (isFraud=1) 
    and transactions posterior to it with either user account, 
    email address or billing address directly linked to these attributes as fraud too. 
    If none of above is reported and found beyond 120 days, 
    then we define as legit transaction (isFraud=0).
"""
transaction_columns = [
    'TransactionID'
    ,'isFraud'
    ,'TransactionDT'
    ,'TransactionAmt'
    ,'ProductCD'
    ,'card1'
    ,'card2'
    ,'card3'
    ,'card4'
    ,'card5'
    ,'addr1'
    ,'addr2'
    ,'P_emaildomain'
    ,'R_emaildomain'
    ,
    
]
df_transaction = transaction_dataset[transaction_columns]
df_transaction

In [ ]:
df_merge = df_transaction.merge(df_identity, on='TransactionID', how='left')
df_merge

In [ ]:
columns = ['isFraud'
    #,'TransactionDT'
    ,'ProductCD'
    ,'card4'
    ,'id_15'
    ,'id_16'
    ,'id_35'
    ,'id_36'
    ,'id_37'
    ,'id_38'
    ,'DeviceType'
    ,'DeviceInfo'
    ,'id_31'
    ,'id_30'
    ,'id_28'
    ,'id_29']

for column in columns:
    print(df_merge[column].value_counts())

In [ ]:
# UserID = combinação de card1, addr1, and D1
transaction_dataset['UserID'] = transaction_dataset['card1'].astype(str) + transaction_dataset['addr1'].astype(str) + transaction_dataset['D1'].astype(str)
transaction_dataset['UserID'] = transaction_dataset['UserID'].str.replace('nan', '0').str.replace('.', '')

# Manter apenas as colunas necessárias
transaction_dataset = transaction_dataset[['UserID','TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt', 'ProductCD', 'card4', 'card6', 'addr1', 'addr2', 'P_emaildomain']]

In [ ]:
identity_dataset = identity_dataset[['TransactionID', 'id_30', 'id_31', 'DeviceType', 'DeviceInfo']]
identity_dataset.rename(columns={'id_30': 'SisOp', 'id_31': 'Browser'}, inplace=True)

In [ ]:
# Normalizar a coluna SisOp
def normalize_sisop_column(serie):
    os_mapping = {
        'ios': 'iOS',
        'android': 'Android',
        'mac': 'Mac',
        'windows': 'Windows',
        'linux': 'Linux'
    }

    def normalize_os(value):
        value = str(value).lower()
        for key in os_mapping:
            if key in value:
                return os_mapping[key]
        return 'Other'
    
    return serie.apply(normalize_os)

identity_dataset['SisOp'] = normalize_sisop_column(identity_dataset['SisOp'])

In [ ]:
# Normalizar a coluna Browser
def normalize_browser_column(serie):
    browser_mapping = {
        'chrome': 'Chrome',
        'firefox': 'Firefox',
        'safari': 'Safari',
        'ie': 'Internet Explorer',
        'edge': 'Edge',
        'samsung': 'Samsung',
        'opera': 'Opera'
    }

    def normalize_browser(value):
        value = str(value).lower()
        for key in browser_mapping:
            if key in value:
                return browser_mapping[key]
        return 'Other'
    
    return serie.apply(normalize_browser)

identity_dataset['Browser'] = normalize_browser_column(identity_dataset['Browser'])


In [ ]:
# Contagem de transações por usuario
user_transactions_count = transaction_dataset.groupby('UserID').size().reset_index(name='transaction_count')
user_transactions_count.sort_values(by='transaction_count', ascending=False).head(5)

In [ ]:
browser_transactions_count = identity_dataset.groupby('Browser').size().reset_index(name='transaction_count')
browser_transactions_count.sort_values(by='transaction_count', ascending=False).head()

In [ ]:
# Merge dos datasets
full_dataset = pd.merge(transaction_dataset, identity_dataset, on='TransactionID', how='inner')

In [ ]:
# Tamanho dos datasets
print('Transaction Dataset:', transaction_dataset.shape)
print('Identity Dataset:', identity_dataset.shape)
print('Full Dataset:', full_dataset.shape)

In [ ]:
print(f"Quantas transações fraudulentas existem no dataset mergeado?\n{full_dataset.isFraud.value_counts().sort_values(ascending=False)}")